In [1]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
channels = [f'ch_{i}' for i in range(1,15)]

In [7]:
def cal_wavelet_coherence(eeg_data):
    # Assuming you have preprocessed EEG data: eeg_data with shape (num_channels, num_samples)

    # Define wavelet parameters
    wavelet = 'morl'
    central_frequency = 30  # Choose an appropriate central frequency

    # Perform wavelet transform for each channel
    wavelet_coeffs = []
    for channel_data in channels:

        coeffs, _ = pywt.cwt(eeg_data[channel_data], np.arange(1, 45 + 1), wavelet)
        wavelet_coeffs.append(coeffs)

    wavelet_coeffs = np.array(wavelet_coeffs)  # Shape: (num_channels, num_scales, num_samples)

    # Calculate wavelet coherence
    num_channels = wavelet_coeffs.shape[0]
    coherence_matrix = np.zeros((num_channels, num_channels))

    for i in range(num_channels):
        for j in range(i + 1, num_channels):
            cross_spectrum = np.multiply(wavelet_coeffs[i], np.conj(wavelet_coeffs[j]))
            coherence = np.abs(np.mean(cross_spectrum, axis=1)) ** 2 / (np.mean(np.abs(wavelet_coeffs[i]) ** 2, axis=1) 
                                                                        * np.mean(np.abs(wavelet_coeffs[j]) ** 2, axis=1))
            coherence_matrix[i, j] = coherence_matrix[j, i] = np.mean(coherence)
    return  coherence_matrix
    # coherence_matrix now contains the wavelet coherence values between each pair of channels



In [9]:
import os
directory = "clean_data/"   #first normalize then clean
for filename in os.listdir(directory):
    data = pd.read_csv(f'{directory}/{filename}')
    label = data['label'][0]
    wav_coh = cal_wavelet_coherence(data.drop(['label'] , axis = 1))
    wav_coh = pd.DataFrame(wav_coh)
    wav_coh['label'] = label
    wav_coh.to_csv(f'wavelete_cohrence/{filename[:-4]}.csv',index = False)